# JuliaPatents + Lens.org demo

(c) 2023 Alex Seitz <ei271@uni-heidelberg.de>

JuliaPatents is a Julia package ecosystem currently in development for patent data analysis.
It currently provides support for [Lens.org](https://lens.org) as a data source,
and will eventually also support [PatStat](https://www.epo.org/searching-for-patents/business/patstat.html).

## Components

The ecosystem consists of the following packages:
- *PatentsBase* for general interface functions
- *PatentsLandscapes* for analysis functions and custom taxonomies
- *PatentsLens* for use of the Lens.org data source

## Scope of this demo




In [3]:
using Pkg
Pkg.add("PatentsBase")
Pkg.add("PatentsLandscapes")
Pkg.add("PatentsLens")

    Updating registry at `~/.julia/registries/JuliaPatents`


    Updating git-repo `git@github.com:JuliaPatents/Registry.git`


    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...


  No Changes to `~/code/PatentsDemo/Project.toml`
  No Changes to `~/code/PatentsDemo/Manifest.toml`


   Resolving package versions...
  No Changes to `~/code/PatentsDemo/Project.toml`
  No Changes to `~/code/PatentsDemo/Manifest.toml`


   Resolving package versions...
  No Changes to `~/code/PatentsDemo/Project.toml`
  No Changes to `~/code/PatentsDemo/Manifest.toml`


In [2]:
using PatentsBase, PatentsLens, PatentsLandscapes